In [ ]:
import pandas as pd

df = pd.read_csv("../data/shrine.csv")
df = df[["id", "name", "name_en", "deity", "deity_en", "lat", "lng", "note_summary_en"]]
df.head()

In [ ]:
import folium

m = folium.Map(location=[35.63, 139.72], zoom_start=13)

for _, row in df.iterrows():
    popup = f"<b>{row['name_en']}</b><br>{row['note_summary_en']}"
    folium.Marker([row["lat"], row["lng"]], popup=popup).add_to(m)

m


![Shrine Network Map](../public/images/shrine_map_meguro-river.PNG)


In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

G = nx.Graph()

# ノード追加
for _, row in df.iterrows():
    G.add_node(row["name_en"], lat=row["lat"], lng=row["lng"], deity=row["deity_en"])

# 神社データフレーム df の中から、すべての神社のペアを作成（重複なし）
for i in range(len(df)):
    for j in range(i+1, len(df)):
      # i番目の神社が祀る神々の英語表記（カンマ区切り）を分割し、集合（set）にする
        set_i = set(df["deity_en"][i].split(","))
        # j番目の神社が祀る神々も同様に集合にする
        set_j = set(df["deity_en"][j].split(","))
        # 共通部分があるか
        if set_i & set_j:
          # もし共通の神がいたら、両神社をエッジでつなぐ（グラフに辺を追加）
            G.add_edge(df["name_en"][i], df["name_en"][j])

# 可視化
plt.figure(figsize=(10, 8))
nx.draw(G, with_labels=True, node_color="skyblue", edge_color="gray", font_size=8)
plt.title("Shrine Network (Shared Deities)")
plt.show()


![Shrine Network](../public/images/shrine-network.png)
